# READ KOFAX EXPORT DATA (DREAM)

Author:   Andreas Barth, SF6-S-OG  
Version:  4, 12.10.2020  

Purpose:  Script to read the email informations (doctypes, body, subject, batchID, pagecount, etc.) out of Export Scripts (File Structure) provided by KOFAX export.


ChangeLog:
V4: Add Reading of new Indexinformation ("AutoClassification")
V3: Change Reading-Process: read Indexfile using List-Comprehensions instead of Regex.

Platform:   HSDAP ONLY! ... (Raw Data is not anonymized yet)


Steps:
  
0. Imports & Functions
1. Execute Reading from File Structure
2. Blacklist Check, Cleaning, Anonymization & Preprocessing


### 0. Imports & Functions

In [11]:
import pandas as pd
import numpy as np
import datetime as dt
import os, re, string, sys
from IPython.core.interactiveshell import InteractiveShell
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt

# ==================== BMW-Bank Funktionen ============================
workDir = os.getcwd()
os.chdir('/home/q506010/0_Packages')
from BmwBankTools.downloadTools import * 
from BmwBankTools.cleanEmails import * 
os.chdir(workDir)

# ==========================================================
%matplotlib inline
pd.set_option('display.max_colwidth', 300)
np.random.seed(4711)
InteractiveShell.ast_node_interactivity = "all"
plt.style.use('ggplot')
#===========================================================
print(sys.version, sys.getdefaultencoding())
os.getcwd()

3.7.7 (default, May  7 2020, 21:25:33) 
[GCC 7.3.0] utf-8


'/home/q506010/1_Kofax_Prod/Learning3010'

***
### 1. Read KOFAX exported data from file structure on disk (HSDAP)
#### 1.1 Create Listing with filepath of each file to be read into system

In [2]:
os.listdir("/home/q506010/1_Kofax_Prod")

['Learning2509',
 '.ipynb_checkpoints',
 'Recognition Time and Effectiven.csv',
 'zipfiles',
 'Learning0910',
 'Learning1610',
 'Learning2310',
 'Learning3010']

In [3]:
workDir = "/home/q506010/1_Kofax_Prod/Learning3010/"  # Learning0910         #/1_Kofax_Mtest/ML/ frühere Massentest Daten im übergeordneten Ordner über /ML/"
os.chdir(workDir)

searchstring = ".txt"
fileList = [os.path.join(dirpath, filename) for dirpath, dirname, files in os.walk(workDir) for filename in files if filename.endswith(searchstring)]
print("# Files: ", len(fileList)); fileList[:6]

# Files:  39392


['/home/q506010/1_Kofax_Prod/Learning3010/Banking/759575/2043206/001F2D46.txt',
 '/home/q506010/1_Kofax_Prod/Learning3010/Banking/759575/2043206/001F2D46_index.txt',
 '/home/q506010/1_Kofax_Prod/Learning3010/Banking/759575/2043206/.ipynb_checkpoints/001F2D46-checkpoint.txt',
 '/home/q506010/1_Kofax_Prod/Learning3010/Banking/759575/2043206/.ipynb_checkpoints/001F2D46_index-checkpoint.txt',
 '/home/q506010/1_Kofax_Prod/Learning3010/Banking/759588/2043235/001F2D63.txt',
 '/home/q506010/1_Kofax_Prod/Learning3010/Banking/759588/2043235/001F2D63_index.txt']

In [4]:
# Create empty Pandas data frames to collect the content from body files and index files   
# DF for text files with bodies
cols = "file_body rawBody".upper().split()
df_BODY = pd.DataFrame(columns=cols)
# DF for indexfiles
cols = "file_index indexString".upper().split()  
df_INDEX = pd.DataFrame(columns=cols)

# helpers
ERRORS = []
START_TIME = dt.datetime.now()
COUNT = 0

# Reading files from Listing
for filepath in fileList:
    COUNT += 1
    fn = filepath.split("/")[-1].split(".")[0]
    with open(filepath,"r", encoding="utf-8") as f:
        content = f.read()
        
    # Read Classification Infos from Indexfiles
    if "_index" in fn:
        try:
            idx = fn.split("_")[0]             # Gleichen Index verwenden wie für das Bodyfile (Emailtext)
            df_INDEX.loc[idx,:] = filepath.split("/")[-1], content 
        except:
            ERRORS.append(filepath)
        
    # Read Text from Textfiles
    else:
        idx = fn
        df_BODY.loc[idx,:] = filepath.split("/")[-1], content 

df = pd.concat([df_INDEX, df_BODY], axis=1)

df.INDEXSTRING = df.INDEXSTRING.str.replace("\ufeff", "")

DURATION = dt.datetime.now() - START_TIME
print(f"{len(ERRORS)} Indexfiles created errors and were not read into dataframe")
print(f"Processing of {COUNT} files took {DURATION.seconds} seconds")

# df.head(3)
df.shape
dfSIK = df.copy()

0 Indexfiles created errors and were not read into dataframe
Processing of 39392 files took 125 seconds


(19696, 4)

#### Read all necessary information from INDEXSTRING

In [25]:
df = dfSIK.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19696 entries, 001F2D46 to 001F2D46-checkpoint
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   FILE_INDEX   19695 non-null  object
 1   INDEXSTRING  19695 non-null  object
 2   FILE_BODY    19696 non-null  object
 3   RAWBODY      19696 non-null  object
dtypes: object(4)
memory usage: 769.4+ KB


In [28]:
df = dfSIK.copy()
df.INDEXSTRING = df.INDEXSTRING.astype('string')
df.info()

df.INDEXSTRING.isna().sum()

# df["BATCHKLASSE"] = df.INDEXSTRING.apply(lambda x: [*x.split(',')][0])

<class 'pandas.core.frame.DataFrame'>
Index: 19696 entries, 001F2D46 to 001F2D46-checkpoint
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   FILE_INDEX   19695 non-null  object
 1   INDEXSTRING  19695 non-null  string
 2   FILE_BODY    19696 non-null  object
 3   RAWBODY      19696 non-null  object
dtypes: object(3), string(1)
memory usage: 769.4+ KB


1

In [24]:
df = dfSIK.copy()

# df.INDEXSTRING = df.INDEXSTRING.apply(str)


df["BATCHKLASSE"] = df.INDEXSTRING.apply(lambda x: [*x.split(',')][0])  #.replace('"','')
# df["BATCHCONTENT"]= df.INDEXSTRING.apply(lambda x: [*x.split(',')][1].replace('"',''))
# df.BATCHKLASSE.head()
df["test1"] = df.INDEXSTRING.apply(lambda x: [*x.split(',')])
df["t2"]    = df.INDEXSTRING.apply(lambda x: x.split(',')[0]).str.strip('"')
df[["test1", "t2"]].head()


# df.info()
# 
# df["STRING"] = df.INDEXSTRING.apply(lambda x:     [*x.split(',')])

AttributeError: 'float' object has no attribute 'split'

In [ ]:
df.STRING.apply(lambda x: x)

In [ ]:
df["BATCHCONTENT"]     = df.INDEXSTRING.apply(lambda x:     x.split(',')[2].replace('"',''))

In [ ]:
df.BATCHKLASSE.value_counts()

In [ ]:
# Create columns using the information provided in the indexstring
df["BATCHKLASSE"]      = df.INDEXSTRING.apply(lambda x:     [*x.split(',')][0].replace('"',''))
df["BATCHCONTENT"]     = df.INDEXSTRING.apply(lambda x:     [*x.split(',')][1].replace('"',''))
df["BATCHID"]          = df.INDEXSTRING.apply(lambda x: int([*x.split(',')][3].replace('"','')))
df["DOCID"]            = df.INDEXSTRING.apply(lambda x: int([*x.split(',')][5].replace('"','')))
df["DOCTYPE"]          = df.INDEXSTRING.apply(lambda x:     [*x.split(',')][7].replace('"',''))
df["CONFIDENCE"]       = df.INDEXSTRING.apply(lambda x:     [*x.split(',')][9].replace('"',''))
# df["AUTOCLASS"]        = df.INDEXSTRING.apply(lambda x:     [*x.split(',')][19].replace('"',''))
df["PAGECOUNT"]        = df.INDEXSTRING.apply(lambda x:     [*x.split(',')][11]).str.strip('"')
df["DOCCOUNT"]         = df.INDEXSTRING.apply(lambda x:     [*x.split(',')][13]).str.strip('"')
df["INPUTCHANNEL"]     = df.INDEXSTRING.apply(lambda x:     [*x.split(',')][15].replace('"',''))
df["SOURCESYSTEM"]     = df.INDEXSTRING.apply(lambda x:     [*x.split(',')][17].replace('"',''))
df["NBR_DOCTYPES"]     = df.CONFIDENCE.apply(lambda x: len(x))

# Convert dtypes to numeric (int) and boolean (T/F)
df.PAGECOUNT = pd.to_numeric(df.PAGECOUNT, errors='coerce').astype("Int64")
df.PAGECOUNT = df.PAGECOUNT.fillna(df.PAGECOUNT.median())
df.DOCCOUNT = pd.to_numeric(df.DOCCOUNT, errors='coerce').astype("Int64")
df.DOCCOUNT = df.DOCCOUNT.fillna(df.DOCCOUNT.median())
# df.AUTOCLASS  = pd.to_numeric(df.AUTOCLASS)
# df.dropna(axis=0, subset=["AUTOCLASS"], inplace=True)
# df.AUTOCLASS  = df.AUTOCLASS.astype("bool")

# Split Information contained in CONFIDENCE by | Separator into list
df.CONFIDENCE = df.CONFIDENCE.str.split("|")

# Drop NA records
df = df.dropna(axis=0, subset=["DOCTYPE"])
df = df.dropna(axis=0, subset=["RAWBODY"])
df.shape

***
## 2. Explorative Data Analysis

In [ ]:
dfM = df.copy()

In [ ]:
dtpc = dfM.groupby("DOCTYPE")["PAGECOUNT"].describe(percentiles=[.8,.9,.95,.99]).sort_values("count", ascending=False)
dtpc["PCOUNT"] = (dtpc["mean"] * dtpc["count"]).astype("int")
dtpc["count"]  = dtpc["count"].astype("int")
dtpc[["mean", "std"]] = dtpc[["mean", "std"]].round(1)
print("Verteilung des Auftretens u. der Seitenzahlen der Dokumententypen")
dtpc.sort_values("PCOUNT", ascending=False)[:20]


In [ ]:
n = 30
t = f"Vert. Dokumententypen T{n} nach #Seiten"
_= dtpc.PCOUNT.sort_values(ascending=True)[-n:].plot(kind="barh", figsize=[4,8], color="#5d788f", title=t)

### Clean KOFAX Data for processing on DLP platform

+ Filter for records containing the relevant trainable doctypes only
+ Blacklist-Filtering of data
+ Cleaning & Anonymization of data
+ Limit length of each document to a maximum length value (shortens very long documents)
+ Select columns necessary for DLP (exclude all raw data columns!!!)
+ Write to disk with "\*.pkl" format



#### Filter for records containing the relevant trainable doctypes only

In [ ]:
dataDir  = "/home/q506010/2_ModelData/"
with open(dataDir+'DocTypesTop20(Pages).txt') as f:
    DocTypes = f.readlines()
    f.close()
DocTypes = sorted([dt.replace("\n", '') for dt in DocTypes])

In [ ]:
dfM.shape
dfC = dfM.loc[dfM.DOCTYPE.isin(DocTypes),:].copy()
dfC.DOCTYPE.value_counts()
dfC.shape

In [ ]:
dfM.shape

##### Blacklist Filter

In [ ]:
dfC = dfM.copy()

BLACKLIST = loadBlacklist()
t0 = dt.datetime.now()

# Conditional Check Column
dfC["BL"] = dfC.RAWBODY.apply(lambda x: checkBlacklist(x, BLACKLIST))
dur = dt.datetime.now() - t0
print(f"Blacklist Filter took: {dur.seconds} seconds, identified {dfC.BL.sum()} records overlapping with Blacklist")

dfC = dfC.loc[dfC.BL==False, :]
dfC.shape
dfCSIK = dfC.copy()

##### Cleaning & Anonymization

In [ ]:
# Anzahl der Zeichen
dfC.RAWBODY.str.len().describe(percentiles=[.8, .9, .92, .95, .98, .99])

# Anzahl der Wörter
dfC.RAWBODY.str.split().str.len().describe(percentiles=[.8, .9,.95,.99])

In [ ]:
dfC = dfCSIK.copy()

time0 = dt.datetime.now()
dfC = CleanREPLACE_KOFAX_Export(dfC)

time1 = dt.datetime.now()
dfC.BODY_CLEAN = dfC.BODY_CLEAN.apply(CleanRGX)

textsize = 10_000
dfC.BODY_CLEAN = dfC.BODY_CLEAN.apply(lambda txt: txt[:textsize])

time2 = dt.datetime.now()
dfC.BODY_CLEAN = dfC.BODY_CLEAN.apply(CleanNER)

time3 = dt.datetime.now()
d1 = time3-time2
d2 = time3-time0
d1.seconds; d2.seconds

dfC.BODY_CLEAN.sample(10)

In [ ]:
idx = "0017E075"
dfC.RAWBODY.loc[idx]
len(dfC.RAWBODY.loc[idx]), len(dfC.RAWBODY.loc[idx].split())
dfC.BODY_CLEAN.loc[idx]
len(dfC.BODY_CLEAN.loc[idx]), len(dfC.BODY_CLEAN.loc[idx].split())

In [ ]:
os.getcwd()

In [ ]:
filter_ = dfC.BODY_CLEAN.str.contains("ANONYMIZATION FAILED")
dfC[filter_].shape
dfC.BL.unique()

In [ ]:
dfC.info()

In [ ]:
dfC.DOCTYPE.nunique()
dfC.shape

In [ ]:
dfC.info()

In [ ]:
dataDir  = "/home/q506010/2_ModelData/"
filename = "L3_v0.pkl"

filter_ = dfC.BODY_CLEAN.str.contains("ANONYMIZATION FAILED")
exportDF = dfC.loc[filter_==False].copy()
exportDF = exportDF.iloc[:,[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16]]

exportDF.to_pickle(dataDir+filename)

***
## KOFAX-Kontrollreport einlesen u. mit KOFAX-Export Dataframe matchen

In [ ]:
# # file://europe.bmw.corp/winfs/SF-D-proj/_2_Internal/2018/Projects/PR-010634_DReAM_Dokumente/Auswertung%20Produktion/09%20Septepmber_Digibox_Recognition%20Time%20and%20Effectinveness/csv%20export/

dfDIGIBOX = pd.read_csv('/home/q506010/1_Kofax_Prod/Recognition Time and Effectiven.csv', sep=";", low_memory=False)
dfDIGIBOX.columns = [c.strip().replace(" ","_").upper().replace("\xa0","") for c in dfDIGIBOX.columns.tolist()]

# dtCols = ["DOCUMENT_ARRIVAL_DEAS",
#           "DOCUMENT_SENT_TO_MANUALCLASSIFICATION",
#           "DOCUMENT_MANUALCLASSIFICATION",
#           "DOCUMENT_SENT_TO_MANUAL_EXTRACTION",
#           "DOCUMENT_MANUAL_EXTRACTION",
#           "DOCUMENT_SENT_TO_DMS"]
# for col in dtCols:
#     dfDIGIBOX[col] = pd.to_datetime(dfDIGIBOX[col])

# dfDIGIBOX = dfDIGIBOX.iloc[:,[0,1,2,3,5,15,16,17,19]]
dfDIGIBOX.info()
dfDIGIBOX.head(2)

dfMClass = dfDIGIBOX.loc[dfDIGIBOX.AUTOMATISCH_KLASSIFIZIERT==0,:].copy()
print(f"Nicht automatisch klassifiert {dfMClass.shape[0]} Dokumente")

In [ ]:
dfM = df.merge(dfDIGIBOX, how="left", left_on="BATCHID", right_on="STAPEL_ID", )  # left oder inner Merge
(dfM.STAPEL_ID == dfM.BATCHID).sum()
dfM.info() 
dfM.AUTOMATISCH_KLASSIFIZIERT.value_counts()

## Parkplatz

In [ ]:
a = {1, 2, 3, 4, 5}
b = { 3, 5, 6, 7, 8}
c = b.intersection(a)
c

In [ ]:
# Nested List Comprehension
non_flat = [ [1,2,3], [4,5,6], [7,8] ]
[y for x in non_flat for y in x]

In [ ]:
# Pandas Dataframe Grundgerüst zum "Einsammeln der Dokumenten-Infos"   
# Bodyfiles
cols = "file_body rawBody".upper().split()
df_BODY = pd.DataFrame(columns=cols)
# Indexfiles
cols = "file_index indexString batchID docID docType confidence pageCount docCount inputChannel sourceSystem".upper().split()   # rawBody
df_INDEX = pd.DataFrame(columns=cols)

ERRORS = []
# Reading files from Listing
for file in fileList: #[:1000]:
    fn = file.split("/")[-1].split(".")[0]
    # print(fn)    
    # Read Classification Infos from Indexfiles
    if "_index" in fn:
        with open(file,"r", encoding="utf-8") as f:  #encoding="utf-8"
            content = f.read()
            
        try:
            batchID, docID, docType, confidence, pageCount, documentCount, inputChannel, sourceSystem = readIndexFileInfo(content)
            idx = fn.split("_")[0]             # Gleichen Index verwenden wie für das Bodyfile (Emailtext)
            df_INDEX.loc[idx,:] = file.split("/")[-1], content, batchID, docID, docType, confidence, pageCount, documentCount, inputChannel, sourceSystem

        except:
            ERRORS.append(file)
    
    
    # Read Emailtext from Textfiles
    else:
        with open(file,"r", encoding="utf-8") as f:
            content = f.read()
        idx = fn
        df_BODY.loc[idx,:] = file.split("/")[-1], content 

df = pd.concat([df_INDEX, df_BODY], axis=1)

# Ergänzende Spalten bauen
df.CONFIDENCE = df.CONFIDENCE.str.split("|")

# df["BATCHKLASSE"] = df.INDEXSTRING.apply(lambda text: text.split(",")[0]).str.replace('"', '').str.lstrip("\ufeff")
# df.INDEXSTRING = df.INDEXSTRING.str.replace("\ufeff", "")
# df["BATCHKLASSE"] = df.INDEXSTRING.str.split(",")
# df.BATCHKLASSE = df.BATCHKLASSE.apply(lambda liste: liste[0].replace('"', ''))

print(f"{len(ERRORS)} Indexfiles created errors and were not read into dataframe")
pd.set_option('display.max_colwidth', 30)
df.head(); df.shape

In [ ]:
def readIndexFileInfo(txt):
    
    # Definition of RGX-objects to find dedicated infos in string
    RGX_batchID       = re.compile( r'"{Batch ID}","(\d{6})' )                                  # Capture 6-Digit Batch ID value
    RGX_docID         = re.compile( r'"{Document ID}","(\d{7})' )                               # Capture 7-Digit Document ID value
    RGX_docType       = re.compile( r'"DocumentType","(\w+)"' )                                 # Capture Document Type
    RGX_confidence    = re.compile( r'"ClassificationResultWithConfiden","(.*)","PageCount"' )  # Capture ALL found Document IDs with Confidence Scores
    RGX_pageCount     = re.compile( r'"PageCount","(\d+)"' )                                    # Capture Page Count 
    RGX_documentCount = re.compile( r'"\{?Document\s?Count\}?","(\d+)"' )                       # Capture Document Count
    RGX_inputChannel  = re.compile( r'"{\$InputChannel}","(\w+)"' )                             # Capture Input Channel
    RGX_sourceSystem  = re.compile( r'"{\$sourceSystem}","(\w+)"' )                             # Capture Source System
    
    # Perform search on above RGX-objects and fill variables with results
    MO_BatchID        = RGX_batchID.search(content)        # MO_ := Match-Object
    batchID           = int(MO_BatchID.group(1))
    
    MO_DocID          = RGX_docID.search(content)
    docID             = int(MO_DocID.group(1))
    
    MO_DocType        = RGX_docType.search(content)
    docType           = MO_DocType.group(1)
        
    MO_Confidence     = RGX_confidence.search(content)
    confidence        = MO_Confidence.group(1)
    
    MO_PageCount      = RGX_pageCount.search(content)
    pageCount         = int(MO_PageCount.group(1))
    
    MO_DocumentCount  = RGX_documentCount.search(content)
    documentCount     = int(MO_DocumentCount.group(1))
     
    MO_InputChannel   = RGX_inputChannel.search(content)
    inputChannel      = MO_InputChannel.group(1)
    
    MO_SourceSystem   = RGX_sourceSystem.search(content)
    sourceSystem      = MO_SourceSystem.group(1)
    
    return batchID, docID, docType, confidence, pageCount, documentCount, inputChannel, sourceSystem
    

In [ ]:
def auspacker(inputstring):
    
    result = []
    for tl in inputstring.split("|"):
        score = float(tl.split(";")[1])
        dtype = tl.split(";")[0]
        code = tl.split(";")[2]
        result.append((score, dtype, code))
    
    return result


In [ ]:
def auspacker(liste):
    L1 = [*liste.split("|")]
    L2 = [[float(tl.split(";")[1]), tl.split(";")[0], tl.split(";")[2]] for tl in L1]
    return L2


In [ ]:
n = 30

fig, (ax1, ax2) = plt.subplots(1,2, sharey=True, figsize=(10,10))

d1 = dtpc.PCOUNT.sort_values(ascending=True)[-n:]
d2 = dfM.DOCTYPE.value_counts()[:n].sort_values(ascending=True)

t1 = f"Vert. Dokumententypen T{n} nach #Seiten"
t2 = f"... nach Häufigkeit"

ax1.barh(d1.index, d1, color="#5d788f")
ax1.set_title(t1)

ax2.barh(d2.index, d2, color="g",)
ax2.set_title(t2)

plt.show()

In [ ]:
# Select necessary columns
dfC = dfM.iloc[:, [3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15] ].copy()
dfC.info()